In [2]:
import psycopg2
import sys  
sys.path.append('..')

import numpy as np
import pandas as pd

from scipy.stats import randint as sp_randint

from config import REDSHIFT_CONFIG
from src.features import *
from src.utils import *
from src.validation import *

from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import svm

In [3]:
# reload(sys)
# sys.setdefaultencoding('utf8')

from src.exploration import *

# Establish a connection to the redshift database
conn = create_rs_conn(config=REDSHIFT_CONFIG)
cur = conn.cursor()

In [5]:
verlander_df = get_pitcher_df_for_modeling(cur, pitcher_id = 434378, date_subsetting = False, table = 'all_pitch_data_reclass')
modeling_data = split_test_train(verlander_df, '2012-06-01')

#get the accuracy for using baseline features and initialize the record_keeping dictionary
classifier = svm.LinearSVC(C=1.0, penalty='l1', dual=False, random_state=35)
classifier.fit(modeling_data['train_data'], modeling_data['train_targets'])

# Make predictions on dev data
dev_predictions = classifier.predict(modeling_data['test_data'])

# Return the dev performance score.
acc = round(accuracy_score(modeling_data['test_targets'], dev_predictions),3)

naive = round(sum(modeling_data['test_targets'].values == 'Fastball') / (1.0 * modeling_data['test_targets'].shape[0]),3)

print acc 
print naive

145 rows didn't contain pitch metadata and were removed
229 rows contained at least 1 NaN and were dropped
0.558
0.56


In [6]:
query = '''select distinct(id) table_id
,trim(datname)   db_name
,trim(nspname)   schema_name
,trim(relname)   table_name
from stv_tbl_perm
join pg_class on pg_class.oid = stv_tbl_perm.id
join pg_namespace on pg_namespace.oid = relnamespace
join pg_database on pg_database.oid = stv_tbl_perm.db_id;'''

cur.execute(query)
cur.fetchall()

[(100003, 'template1', 'pg_catalog', 'padb_config_harvest'),
 (100000, 'template1', 'pg_catalog', 'systable_globaldict'),
 (136415, 'dev', 'public', 'all_pitch_data_reclass'),
 (136411, 'dev', 'public', 'pt_reclassified_rules_simplified'),
 (136381, 'dev', 'public', 'pt_reclassified_fa'),
 (136377, 'dev', 'public', 'pt_reclassified'),
 (127218, 'dev', 'public', 'hitchart'),
 (127206, 'dev', 'public', 'game'),
 (127222, 'dev', 'public', 'player'),
 (136413, 'dev', 'public', 'pitch_reclass'),
 (100001, 'template1', 'pg_catalog', 'systable_topology'),
 (100002, 'template1', 'pg_catalog', 'systable_schema'),
 (127214, 'dev', 'public', 'pitch'),
 (127210, 'dev', 'public', 'atbat'),
 (127202, 'dev', 'public', 'full_pitch_data'),
 (136409, 'dev', 'public', 'pt_reclassified_rules'),
 (127346, 'dev', 'public', 'all_pitch_data')]